# Speed Analysis

In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import os
import sumolib
import sys
import optparse
import random

ROOT = Path(os.getcwd()).parent

In [7]:
actual_df = pd.read_csv(ROOT / "data" / "test.csv")
pd.options.plotting.backend = "plotly"

In [8]:
rows = []

for r in sumolib.xml.parse_fast_nested(str(ROOT / "sumo-xml" / "output" / "d_fcd.xml"), "timestep", ["time"], "vehicle", ["id", "speed", "pos", "acceleration"]):
    rows.append(
        {
            **r[0]._asdict(),
            **r[1]._asdict()
        }
    )

In [15]:
r[1]._asdict()

{'id': 'leader', 'speed': '7.73', 'pos': '338.99', 'acceleration': '-2.40'}

In [10]:
sumo_df = pd.DataFrame(rows)
leader_df = sumo_df.query("id == 'leader'")
follower_df = sumo_df.query("id == 'follower'")

In [11]:
vt_fig = go.Figure(data=go.Scatter(x=actual_df['seconds'], y=actual_df['leadvelocity'], name='RW leader'))
vt_fig.add_scatter(x=actual_df['seconds'], y=actual_df['followvelocity'], name='RW follower')

vt_fig.add_scatter(x=leader_df["time"], y=leader_df["speed"], name="sumo leader")
vt_fig.add_scatter(x=follower_df["time"], y=follower_df["speed"], name="sumo follower")

vt_fig.update_layout(
    title="Velocity vs Time (RW vs SUMO base 32)",
    xaxis_title="Time (s)",
    yaxis_title="Velocity (m/s)",
    legend_title="Source",
)

vt_fig.show()

In [12]:
dt_fig = go.Figure(data=go.Scatter(x=actual_df['seconds'], y=actual_df['leadposition'], name='RW leader'))

dt_fig.add_scatter(x=actual_df['seconds'], y=actual_df['followposition'], name='RW follower')
dt_fig.add_scatter(x=leader_df["time"], y=leader_df["pos"], name="sumo leader")
dt_fig.add_scatter(x=follower_df["time"], y=follower_df["pos"], name="sumo follower")

dt_fig.update_layout(
    title="Distance vs Time (RW vs SUMO)",
    xaxis_title="Time (s)",
    yaxis_title="Distance (m)",
    legend_title="Source",
)

dt_fig.show()

In [13]:
at_fig = go.Figure(data=go.Scatter(x=actual_df['seconds'], y=actual_df['leadacceleration'], name='RW leader'))

at_fig.add_scatter(x=actual_df["seconds"], y=actual_df["followacceleration"], name="RW follower")
at_fig.add_scatter(x=leader_df["time"], y=leader_df["acceleration"], name="sumo leader")
at_fig.add_scatter(x=follower_df["time"], y=follower_df["acceleration"], name="sumo follower")

at_fig.update_layout(
    title="Acceleration vs Time (RW vs SUMO)",
    xaxis_title="Time (s)",
    yaxis_title="Acceleration (m/s^2)",
    legend_title="Source",
)

at_fig.show()